In [17]:
from tensorflow import keras
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.python.keras.models import *
from tensorflow.python.keras.activations import *
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.losses import *
from tensorflow.python.keras.optimizers import *
from tensorflow.python.keras.utils import *
from tensorflow.python.keras.metrics import *
from tensorflow.python.keras.datasets import *
from tensorflow.python.keras.callbacks import *
import pydot as pyd
import datetime

In [18]:
TRAIN_FOLDER = "dice/train/"
TEST_FOLDER = "dice/valid/"

In [5]:
folder_list = []
image_list = []

# Preprocessing data
part_of_test = ["d4", "d8"]
for folder in part_of_test:
    for image in os.listdir(TRAIN_FOLDER + folder):
        image_read = cv2.imread(TRAIN_FOLDER + folder + "/" + image)
        #image_reshaped = np.reshape(image_read, (1, 691200))/255.0
        image_list.append(image_read)
        folder_list.append(int(folder[1:]))
image_list = np.reshape(image_list, (3479, 691200))/255.0

In [6]:
folder_test_list = []
image_test_list = []


for folder in part_of_test:
    for image in os.listdir(TEST_FOLDER + folder):
        image_test_read = cv2.imread(TEST_FOLDER + folder + "/" + image)
        #image_test_reshaped = np.reshape(image_test_read, (1, 691200))/255.0
        image_test_list.append(image_test_read)
        folder_test_list.append(int(folder[1:]))
        
image_test_list = np.reshape(image_test_list, (496, 691200))/255.0

In [19]:
def create_mlp(input_dim: int, hidden_layer_count: int, neurons_count_per_hidden_layer: int, output_dim):
    model = Sequential()

    for i in range(hidden_layer_count):
        if i == 0:
            model.add(Dense(neurons_count_per_hidden_layer, activation=tanh, input_dim=input_dim))
        else:
            model.add(Dense(neurons_count_per_hidden_layer,
                            activation=tanh))

    model.add(Dense(output_dim, activation=sigmoid))
    model.compile(optimizer=sgd(), loss=mse, metrics=["accuracy"])

    return model

In [13]:
x_train = image_list
y_train = folder_list
x_test = image_test_list
y_test = folder_test_list


y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

tp_callback = TensorBoard("./log/linear_" + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S"))


model = keras.models.Sequential()
#model.add(keras.layers.Dense(10, activation=keras.activations.sigmoid, input_dim=691200))
#model.compile(loss=keras.losses.mse, optimizer=keras.optimizers.SGD(), metrics=[keras.metrics.categorical_accuracy])
model.add(keras.layers.Dense(10, activation="sigmoid", input_dim=691200))
model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["categorical_accuracy"])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=128, callbacks=[tp_callback])

Train on 3479 samples, validate on 496 samples
Epoch 1/100
3479/3479 [==============================] - 123s 35ms/sample - loss: 1.9448 - categorical_accuracy: 0.5384 - val_loss: 1.9066 - val_categorical_accuracy: 0.5605
Epoch 2/100
3479/3479 [==============================] - 129s 37ms/sample - loss: 1.8828 - categorical_accuracy: 0.5596 - val_loss: 1.8514 - val_categorical_accuracy: 0.5605
Epoch 3/100
3479/3479 [==============================] - 122s 35ms/sample - loss: 1.8200 - categorical_accuracy: 0.5596 - val_loss: 1.7853 - val_categorical_accuracy: 0.5605
Epoch 4/100
3479/3479 [==============================] - 119s 34ms/sample - loss: 1.7578 - categorical_accuracy: 0.5596 - val_loss: 1.7273 - val_categorical_accuracy: 0.5605
Epoch 5/100
3479/3479 [==============================] - 114s 33ms/sample - loss: 1.7025 - categorical_accuracy: 0.5596 - val_loss: 1.6748 - val_categorical_accuracy: 0.5605
Epoch 6/100
3479/3479 [==============================] - 125s 36ms/sample - loss: 1

KeyboardInterrupt: 

In [20]:
x_train = image_list
y_train = folder_list
x_test = image_test_list
y_test = folder_test_list


y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

for l in range(0, 3):
        for power_n in range(2, 5):
            
            model = create_mlp(691200, l, 2 ** power_n, 10)

            model_name = "mlp" + str(l) + "_" + str(2 ** power_n)+"_"+datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

            tp_callback = TensorBoard("./log/" + model_name )

            model.fit(x_train, y_train,
                      epochs=100,
                      batch_size=128,
                      validation_data=(x_test, y_test), callbacks=[tp_callback])

            plot_model(model, "./models/" + model_name + ".png", show_shapes=True, show_layer_names=True)
            model.save("./models/" + model_name)

Train on 3479 samples, validate on 496 samples
Epoch 1/100
3479/3479 [==============================] - 120s 35ms/sample - loss: 0.1077 - acc: 0.5591 - val_loss: 0.1000 - val_acc: 0.5605
Epoch 2/100
3479/3479 [==============================] - 118s 34ms/sample - loss: 0.1000 - acc: 0.5596 - val_loss: 0.1000 - val_acc: 0.5605
Epoch 3/100
3479/3479 [==============================] - 124s 36ms/sample - loss: 0.0943 - acc: 0.5596 - val_loss: 0.0879 - val_acc: 0.5605
Epoch 4/100
3479/3479 [==============================] - 123s 35ms/sample - loss: 0.0881 - acc: 0.5596 - val_loss: 0.0879 - val_acc: 0.5605
Epoch 5/100
3479/3479 [==============================] - 124s 36ms/sample - loss: 0.0881 - acc: 0.5596 - val_loss: 0.0879 - val_acc: 0.5605
Epoch 6/100
3479/3479 [==============================] - 128s 37ms/sample - loss: 0.0881 - acc: 0.5596 - val_loss: 0.0879 - val_acc: 0.5605
Epoch 7/100
3479/3479 [==============================] - 130s 37ms/sample - loss: 0.0881 - acc: 0.5596 - val_loss

Epoch 59/100
3479/3479 [==============================] - 138s 40ms/sample - loss: 0.0881 - acc: 0.5596 - val_loss: 0.0879 - val_acc: 0.5605
Epoch 60/100
3479/3479 [==============================] - 136s 39ms/sample - loss: 0.0881 - acc: 0.5596 - val_loss: 0.0879 - val_acc: 0.5605
Epoch 61/100
3479/3479 [==============================] - 138s 40ms/sample - loss: 0.0881 - acc: 0.5596 - val_loss: 0.0879 - val_acc: 0.5605
Epoch 62/100
3479/3479 [==============================] - 140s 40ms/sample - loss: 0.0881 - acc: 0.5596 - val_loss: 0.0879 - val_acc: 0.5605
Epoch 63/100
3479/3479 [==============================] - 136s 39ms/sample - loss: 0.0881 - acc: 0.5596 - val_loss: 0.0879 - val_acc: 0.5605
Epoch 64/100
3479/3479 [==============================] - 138s 40ms/sample - loss: 0.0881 - acc: 0.5596 - val_loss: 0.0879 - val_acc: 0.5605
Epoch 65/100
3479/3479 [==============================] - 138s 40ms/sample - loss: 0.0881 - acc: 0.5596 - val_loss: 0.0879 - val_acc: 0.5605
Epoch 66/100


ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.